# Pregunta 2

Instalamos pyspark en caso de que sea necesario para correrlo en google colab.

In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=4678ba6a3f01b4db0d10d62cc8cedf8d0c2d279981f4fce3adf54033a1ac6879
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


Creamos contexto

In [2]:
from pyspark import SparkContext

# Contexto
sc = SparkContext()

Funcion que inicializa grafo como RDD en formato: (nodo_source, [nodo_target, peso, peso_acumulado]).

In [3]:
# Inicializa pesos de grafo
def inicializar(G, source):
    G_pesos = [(node[0], [node[1][0], node[1][1], float('inf')]) if node[0] != source else (node[0], [node[1][0], node[1][1], 0]) for node in G]
    return sc.parallelize(G_pesos)

Inicializamos Grafo de prueba chico y lo restructuramos.

In [4]:
# Grafo de prueba
G = [(1, [2, 1]), (1, [3, 4]), (2, [3, 2]), (2, [4, 3]), (3, [4, 1])]
source = 1
G_init = inicializar(G, source)

# Re-estructura llave-valor (nodo, distancia)
distancias = G_init.map(lambda x: (x[0], x[1][2]))

In [5]:
# Si costo no cambia => break
while True:
    actualizaciones = G_init.join(distancias) \
        .flatMap(lambda x: [(x[1][0][0], x[1][0][1] + x[1][1]), (x[0], x[1][1])]) \
        .reduceByKey(min)

    distancias_nuevas = distancias.fullOuterJoin(actualizaciones) \
        .mapValues(lambda x: min(x[0] or float('inf'), x[1] or float('inf')))

    cambio_costo = distancias_nuevas.join(distancias) \
        .filter(lambda x: x[1][0] != x[1][1]) \
        .count()

    distancias = distancias_nuevas

    if cambio_costo == 0:
        break

Printeamos distancias entre nodo source y los caminos mas cortos hacia los otros nodos

In [6]:
nodos_printeados = set()
for nodo, distancia in distancias.filter(lambda x: x[0] != source).collect():
    if nodo not in nodos_printeados:
        print(f"Valor de SSSP desde nodo {source} (Source) hacia nodo {nodo} es {distancia}")
        nodos_printeados.add(nodo)

Valor de SSSP desde nodo 1 (Source) hacia nodo 2 es 1
Valor de SSSP desde nodo 1 (Source) hacia nodo 3 es 3
Valor de SSSP desde nodo 1 (Source) hacia nodo 4 es 4


Ahora probamos algoritmo con nodo del enunciado.

In [7]:
# Grafo de prueba
G = [(1, [2, 10]), (2, [3, 3]), (2, [4, 24]), (3, [2, 1])]
source = 1
G_init = inicializar(G, source)

# Re-estructura llave-valor (nodo, distancia)
distancias = G_init.map(lambda x: (x[0], x[1][2]))

In [8]:
# Si costo no cambia => break
while True:
    actualizaciones = G_init.join(distancias) \
        .flatMap(lambda x: [(x[1][0][0], x[1][0][1] + x[1][1]), (x[0], x[1][1])]) \
        .reduceByKey(min)

    distancias_nuevas = distancias.fullOuterJoin(actualizaciones) \
        .mapValues(lambda x: min(x[0] or float('inf'), x[1] or float('inf')))

    cambio_costo = distancias_nuevas.join(distancias) \
        .filter(lambda x: x[1][0] != x[1][1]) \
        .count()

    distancias = distancias_nuevas

    if cambio_costo == 0:
        break

In [9]:
nodos_printeados = set()
for nodo, distancia in distancias.filter(lambda x: x[0] != source).collect():
    if nodo not in nodos_printeados:
        print(f"Valor de SSSP desde nodo {source} (Source) hacia nodo {nodo} es {distancia}")
        nodos_printeados.add(nodo)

Valor de SSSP desde nodo 1 (Source) hacia nodo 2 es 10
Valor de SSSP desde nodo 1 (Source) hacia nodo 3 es 13
Valor de SSSP desde nodo 1 (Source) hacia nodo 4 es 34
